In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import time

from model import Generator, Discriminator
from config import irgan_config
from data_utils import RecDataset, DataProvider
from evaluation.rec_evaluator import RecEvaluator

In [2]:
epochs = irgan_config.epochs
batch_size = irgan_config.batch_size
epochs_d = irgan_config.epochs_d
epochs_g = irgan_config.epochs_g
emb_dim = irgan_config.emb_dim
eta_G = irgan_config.eta_G
eta_D = irgan_config.eta_D
device = irgan_config.device
weight_decay_g = irgan_config.weight_decay_g
weight_decay_d = irgan_config.weight_decay_d

In [3]:
# Hyper-parameters and datset-specific parameters
rec_dataset = RecDataset(irgan_config.dir_path)
all_users = rec_dataset.get_users()
all_items = rec_dataset.get_items()
num_users = rec_dataset.get_num_users()
num_items = rec_dataset.get_num_items()
bought_mask = rec_dataset.get_bought_mask().to(device)
eval_dict = rec_dataset.get_interaction_records("test")
train_ui = rec_dataset.get_user_item_pairs()

In [4]:
dp = DataProvider(device)
evaluator = RecEvaluator(eval_dict, None, device)

In [5]:
writer_dis = SummaryWriter("runs/IRGAN-SGD-dns-gen/Discriminator")
writer_gen = SummaryWriter("runs/IRGAN-SGD-dns-gen/Generator")

In [6]:
G = Generator(num_users, num_items, emb_dim, bought_mask)
D = Discriminator(num_users, num_items, emb_dim)
G = G.to(device)
D = D.to(device)

loss_D = nn.BCELoss()
optimizer_G = torch.optim.SGD(G.parameters(), lr = eta_G)
optimizer_D = torch.optim.SGD(D.parameters(), lr = eta_D, weight_decay = weight_decay_d)

In [7]:
pretrained_model = torch.load("./pretrained_models/pretrained_model_dns.pkl",map_location=irgan_config.device)
G.load_state_dict(pretrained_model)

<All keys matched successfully>

In [8]:
for epoch in range(epochs):
    print(
            "[Epoch %d/%d]"
            % (epoch+1, epochs)
        )  
    time_epoch_start = time.time()
    
    # ---------------------
    #  Train Discriminator
    # ---------------------   
    D.train()
    G.eval()
    for epoch_d in range(epochs_d):
        time_d_start = time.time()
        if(epoch_d % 5 == 0):
            train_set_d = dp.prepare_data_for_discriminator(G, train_ui, batch_size = batch_size)
        loss_d_epoch = 0
        for users, items, labels in train_set_d:
            dis_score = D(users,items)
            loss_d = loss_D(dis_score, labels)
            loss_d_epoch += loss_d.item()
            optimizer_D.zero_grad()
            loss_d.backward()
            optimizer_D.step()
            
        time_d_end = time.time()
        loss_d_epoch /= len(train_set_d)
        print(
            "\t[Discriminator][Epochs %d/%d] [D epoch loss: %6.5f] [Time:%6.5f] "
            % (epoch_d+1, epochs_d, loss_d_epoch, time_d_end - time_d_start)
        )
        writer_dis.add_scalar('Loss/Discriminator', loss_d_epoch, epoch*epochs_d + epoch_d)
        with torch.no_grad():
            res = evaluator.top_k_evaluation(D, [3,5,10])    
            ndcg3, precision3, hit3, map3, mrr3 = res[0]
            ndcg5, precision5, hit5, map5, mrr5 = res[1]
            ndcg10, precision10, hit10, map10, mrr10 = res[2]
            writer_dis.add_scalar('Metrics/NDCG@3', ndcg3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/NDCG@5', ndcg5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/NDCG@10', ndcg10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@3', precision3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@5', precision5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@10', precision10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@3', hit3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@5', hit5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@10', hit10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/MAP', map10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/MRR', mrr10, epoch*epochs_d + epoch_d)
        
    # --------------------- 
    #  Train Generator
    #
    # For generator
    #   Generate K user-item pairs
    #   Leveraging Policy Gradient to update parameters of generator
    #
    # --------------------- 
    D.eval()
    G.train()
    for epoch_g in range(epochs_g):
        time_g_start = time.time()
        train_set_g = dp.prepare_data_for_generator(all_users, batch_size)
        loss_g_epoch = 0
        for fake_users, in train_set_g:
            fake_items, fake_probs, fake_p_n = G.sample_items_for_users(fake_users, k = 256,\
                                                                        temperature=1, lambda_bought=0.2)
            fake_users = fake_users.view(-1,1).expand_as(fake_items).contiguous()
            fake_users = fake_users.view(-1)
            fake_items = fake_items.view(-1)
            fake_probs = fake_probs.view(-1)
            fake_p_n = fake_p_n.view(-1)
            
            log_fake_probs = torch.log(fake_probs.clamp(1e-8))
            fake_probs = fake_probs.detach()
            fake_p_n = fake_p_n.detach()
            
            reward = (2 * D(fake_users, fake_items) - 1).detach()*(fake_probs/fake_p_n)
            loss_g = -torch.mean(log_fake_probs*reward)
            loss_g_epoch += loss_g
            
            optimizer_G.zero_grad()  
            loss_g.backward() 
            optimizer_G.step()
        time_g_end = time.time()
        print(
            "\t[Generator][Epochs %d/%d] [G epoch loss: %6.5f] [Time:%6.5f]"
            % (epoch_g + 1, epochs_g, loss_g_epoch, time_g_end - time_g_start )
        ) 
        writer_gen.add_scalar('Loss/Generator', loss_g_epoch, epoch*epochs_g + epoch_g)
        with torch.no_grad():
            res = evaluator.top_k_evaluation(G, [3,5,10])    
            ndcg3, precision3, hit3, map3, mrr3 = res[0]
            ndcg5, precision5, hit5, map5, mrr5 = res[1]
            ndcg10, precision10, hit10, map10, mrr10 = res[2]
            writer_gen.add_scalar('Metrics/NDCG@3', ndcg3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/NDCG@5', ndcg5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/NDCG@10', ndcg10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@3', precision3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@5', precision5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@10', precision10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@3', hit3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@5', hit5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@10', hit10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/MAP', map10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/MRR', mrr10, epoch*epochs_g + epoch_g)
    time_epoch_end = time.time()
writer_dis.close()    
writer_gen.close()

[Epoch 1/200]
	[Discriminator][Epochs 1/100] [D epoch loss: 0.67377] [Time:8.41566] 
	[Discriminator][Epochs 2/100] [D epoch loss: 0.63962] [Time:8.26835] 
	[Discriminator][Epochs 3/100] [D epoch loss: 0.61129] [Time:8.48296] 
	[Discriminator][Epochs 4/100] [D epoch loss: 0.58747] [Time:8.46446] 
	[Discriminator][Epochs 5/100] [D epoch loss: 0.56718] [Time:7.89676] 
	[Discriminator][Epochs 6/100] [D epoch loss: 0.54961] [Time:8.29652] 
	[Discriminator][Epochs 7/100] [D epoch loss: 0.53432] [Time:8.29282] 
	[Discriminator][Epochs 8/100] [D epoch loss: 0.52085] [Time:8.42684] 
	[Discriminator][Epochs 9/100] [D epoch loss: 0.50888] [Time:9.93051] 
	[Discriminator][Epochs 10/100] [D epoch loss: 0.49814] [Time:9.98919] 
	[Discriminator][Epochs 11/100] [D epoch loss: 0.48832] [Time:10.00028] 
	[Discriminator][Epochs 12/100] [D epoch loss: 0.47952] [Time:10.47760] 
	[Discriminator][Epochs 13/100] [D epoch loss: 0.47148] [Time:10.10077] 
	[Discriminator][Epochs 14/100] [D epoch loss: 0.46411] 

	[Generator][Epochs 16/50] [G epoch loss: -9.06977] [Time:0.03019]
	[Generator][Epochs 17/50] [G epoch loss: -9.05219] [Time:0.03145]
	[Generator][Epochs 18/50] [G epoch loss: -8.93649] [Time:0.03777]
	[Generator][Epochs 19/50] [G epoch loss: -8.94785] [Time:0.03707]
	[Generator][Epochs 20/50] [G epoch loss: -8.86233] [Time:0.03044]
	[Generator][Epochs 21/50] [G epoch loss: -8.84453] [Time:0.03043]
	[Generator][Epochs 22/50] [G epoch loss: -8.91119] [Time:0.03016]
	[Generator][Epochs 23/50] [G epoch loss: -8.83825] [Time:0.03635]
	[Generator][Epochs 24/50] [G epoch loss: -8.87433] [Time:0.03370]
	[Generator][Epochs 25/50] [G epoch loss: -8.83369] [Time:0.04638]
	[Generator][Epochs 26/50] [G epoch loss: -8.81983] [Time:0.03345]
	[Generator][Epochs 27/50] [G epoch loss: -8.76414] [Time:0.03700]
	[Generator][Epochs 28/50] [G epoch loss: -8.77395] [Time:0.03223]
	[Generator][Epochs 29/50] [G epoch loss: -8.83780] [Time:0.03331]
	[Generator][Epochs 30/50] [G epoch loss: -8.93708] [Time:0.02

	[Discriminator][Epochs 82/100] [D epoch loss: 0.32227] [Time:10.01749] 
	[Discriminator][Epochs 83/100] [D epoch loss: 0.32208] [Time:10.04844] 
	[Discriminator][Epochs 84/100] [D epoch loss: 0.32188] [Time:8.77966] 
	[Discriminator][Epochs 85/100] [D epoch loss: 0.32168] [Time:8.85128] 
	[Discriminator][Epochs 86/100] [D epoch loss: 0.32145] [Time:8.80697] 
	[Discriminator][Epochs 87/100] [D epoch loss: 0.32126] [Time:8.73439] 
	[Discriminator][Epochs 88/100] [D epoch loss: 0.32107] [Time:9.09572] 
	[Discriminator][Epochs 89/100] [D epoch loss: 0.32088] [Time:10.07904] 
	[Discriminator][Epochs 90/100] [D epoch loss: 0.32069] [Time:10.21097] 
	[Discriminator][Epochs 91/100] [D epoch loss: 0.32056] [Time:9.32913] 
	[Discriminator][Epochs 92/100] [D epoch loss: 0.32038] [Time:9.09236] 
	[Discriminator][Epochs 93/100] [D epoch loss: 0.32020] [Time:8.91794] 
	[Discriminator][Epochs 94/100] [D epoch loss: 0.32002] [Time:9.90672] 
	[Discriminator][Epochs 95/100] [D epoch loss: 0.31984] [Tim

	[Discriminator][Epochs 49/100] [D epoch loss: 0.33075] [Time:10.39477] 
	[Discriminator][Epochs 50/100] [D epoch loss: 0.33059] [Time:10.49754] 
	[Discriminator][Epochs 51/100] [D epoch loss: 0.33051] [Time:10.33117] 
	[Discriminator][Epochs 52/100] [D epoch loss: 0.33035] [Time:10.74444] 
	[Discriminator][Epochs 53/100] [D epoch loss: 0.33020] [Time:10.60419] 
	[Discriminator][Epochs 54/100] [D epoch loss: 0.33004] [Time:10.30463] 
	[Discriminator][Epochs 55/100] [D epoch loss: 0.32988] [Time:10.34979] 
	[Discriminator][Epochs 56/100] [D epoch loss: 0.32971] [Time:10.76282] 
	[Discriminator][Epochs 57/100] [D epoch loss: 0.32956] [Time:9.26518] 
	[Discriminator][Epochs 58/100] [D epoch loss: 0.32941] [Time:9.44460] 
	[Discriminator][Epochs 59/100] [D epoch loss: 0.32926] [Time:10.06113] 
	[Discriminator][Epochs 60/100] [D epoch loss: 0.32911] [Time:8.91648] 
	[Discriminator][Epochs 61/100] [D epoch loss: 0.32906] [Time:9.32624] 
	[Discriminator][Epochs 62/100] [D epoch loss: 0.32891]

	[Discriminator][Epochs 16/100] [D epoch loss: 0.34130] [Time:9.79543] 
	[Discriminator][Epochs 17/100] [D epoch loss: 0.34113] [Time:9.91458] 
	[Discriminator][Epochs 18/100] [D epoch loss: 0.34096] [Time:9.98236] 
	[Discriminator][Epochs 19/100] [D epoch loss: 0.34080] [Time:9.83842] 
	[Discriminator][Epochs 20/100] [D epoch loss: 0.34063] [Time:9.87827] 
	[Discriminator][Epochs 21/100] [D epoch loss: 0.34057] [Time:9.41543] 
	[Discriminator][Epochs 22/100] [D epoch loss: 0.34041] [Time:9.27377] 
	[Discriminator][Epochs 23/100] [D epoch loss: 0.34025] [Time:9.10271] 
	[Discriminator][Epochs 24/100] [D epoch loss: 0.34009] [Time:10.22386] 
	[Discriminator][Epochs 25/100] [D epoch loss: 0.33993] [Time:10.11966] 
	[Discriminator][Epochs 26/100] [D epoch loss: 0.33981] [Time:9.62789] 
	[Discriminator][Epochs 27/100] [D epoch loss: 0.33966] [Time:10.36330] 
	[Discriminator][Epochs 28/100] [D epoch loss: 0.33950] [Time:10.46236] 
	[Discriminator][Epochs 29/100] [D epoch loss: 0.33935] [Tim

...

	[Discriminator][Epochs 91/100] [D epoch loss: 0.37349] [Time:5.37403] 
	[Discriminator][Epochs 92/100] [D epoch loss: 0.37347] [Time:5.41119] 
	[Discriminator][Epochs 93/100] [D epoch loss: 0.37346] [Time:5.48226] 
	[Discriminator][Epochs 94/100] [D epoch loss: 0.37344] [Time:5.39340] 
	[Discriminator][Epochs 95/100] [D epoch loss: 0.37343] [Time:5.41246] 
	[Discriminator][Epochs 96/100] [D epoch loss: 0.37354] [Time:5.37015] 
	[Discriminator][Epochs 97/100] [D epoch loss: 0.37353] [Time:5.38857] 
	[Discriminator][Epochs 98/100] [D epoch loss: 0.37352] [Time:5.40205] 
	[Discriminator][Epochs 99/100] [D epoch loss: 0.37350] [Time:5.39644] 
	[Discriminator][Epochs 100/100] [D epoch loss: 0.37349] [Time:5.44171] 
	[Generator][Epochs 1/50] [G epoch loss: -15.27328] [Time:0.03699]
	[Generator][Epochs 2/50] [G epoch loss: -15.42471] [Time:0.03788]
	[Generator][Epochs 3/50] [G epoch loss: -15.63601] [Time:0.03807]
	[Generator][Epochs 4/50] [G epoch loss: -15.46354] [Time:0.03764]
	[Generator

	[Discriminator][Epochs 58/100] [D epoch loss: 0.37437] [Time:5.33304] 
	[Discriminator][Epochs 59/100] [D epoch loss: 0.37435] [Time:5.40477] 
	[Discriminator][Epochs 60/100] [D epoch loss: 0.37434] [Time:5.43311] 
	[Discriminator][Epochs 61/100] [D epoch loss: 0.37413] [Time:5.51642] 
	[Discriminator][Epochs 62/100] [D epoch loss: 0.37411] [Time:5.52134] 
	[Discriminator][Epochs 63/100] [D epoch loss: 0.37410] [Time:5.48819] 
	[Discriminator][Epochs 64/100] [D epoch loss: 0.37409] [Time:5.42885] 
	[Discriminator][Epochs 65/100] [D epoch loss: 0.37407] [Time:5.55649] 
	[Discriminator][Epochs 66/100] [D epoch loss: 0.37394] [Time:6.43671] 
	[Discriminator][Epochs 67/100] [D epoch loss: 0.37392] [Time:5.50503] 
	[Discriminator][Epochs 68/100] [D epoch loss: 0.37391] [Time:5.48280] 
	[Discriminator][Epochs 69/100] [D epoch loss: 0.37390] [Time:5.49699] 
	[Discriminator][Epochs 70/100] [D epoch loss: 0.37388] [Time:5.49680] 
	[Discriminator][Epochs 71/100] [D epoch loss: 0.37379] [Time:5.

	[Discriminator][Epochs 25/100] [D epoch loss: 0.37431] [Time:5.49235] 
	[Discriminator][Epochs 26/100] [D epoch loss: 0.37453] [Time:5.82226] 
	[Discriminator][Epochs 27/100] [D epoch loss: 0.37452] [Time:5.98878] 
	[Discriminator][Epochs 28/100] [D epoch loss: 0.37450] [Time:5.78700] 
	[Discriminator][Epochs 29/100] [D epoch loss: 0.37449] [Time:5.46630] 
	[Discriminator][Epochs 30/100] [D epoch loss: 0.37448] [Time:5.47214] 
	[Discriminator][Epochs 31/100] [D epoch loss: 0.37448] [Time:4.58735] 
	[Discriminator][Epochs 32/100] [D epoch loss: 0.37447] [Time:5.82856] 
	[Discriminator][Epochs 33/100] [D epoch loss: 0.37446] [Time:5.35565] 
	[Discriminator][Epochs 34/100] [D epoch loss: 0.37444] [Time:5.55792] 
	[Discriminator][Epochs 35/100] [D epoch loss: 0.37443] [Time:5.40590] 
	[Discriminator][Epochs 36/100] [D epoch loss: 0.37408] [Time:5.40709] 
	[Discriminator][Epochs 37/100] [D epoch loss: 0.37406] [Time:5.49465] 
	[Discriminator][Epochs 38/100] [D epoch loss: 0.37405] [Time:5.

	[Generator][Epochs 42/50] [G epoch loss: -16.18331] [Time:0.03720]
	[Generator][Epochs 43/50] [G epoch loss: -16.34406] [Time:0.03733]
	[Generator][Epochs 44/50] [G epoch loss: -16.22718] [Time:0.03735]
	[Generator][Epochs 45/50] [G epoch loss: -16.33213] [Time:0.03604]
	[Generator][Epochs 46/50] [G epoch loss: -16.11756] [Time:0.03565]
	[Generator][Epochs 47/50] [G epoch loss: -16.07107] [Time:0.03707]
	[Generator][Epochs 48/50] [G epoch loss: -16.16774] [Time:0.03768]
	[Generator][Epochs 49/50] [G epoch loss: -16.17151] [Time:0.03657]
	[Generator][Epochs 50/50] [G epoch loss: -16.29888] [Time:0.03467]
[Epoch 200/200]
	[Discriminator][Epochs 1/100] [D epoch loss: 0.37374] [Time:5.22634] 
	[Discriminator][Epochs 2/100] [D epoch loss: 0.37373] [Time:5.30305] 
	[Discriminator][Epochs 3/100] [D epoch loss: 0.37371] [Time:5.18067] 
	[Discriminator][Epochs 4/100] [D epoch loss: 0.37370] [Time:5.45789] 
	[Discriminator][Epochs 5/100] [D epoch loss: 0.37369] [Time:5.71279] 
	[Discriminator][

	[Generator][Epochs 7/50] [G epoch loss: -16.45176] [Time:0.03661]
	[Generator][Epochs 8/50] [G epoch loss: -16.26221] [Time:0.03306]
	[Generator][Epochs 9/50] [G epoch loss: -16.41906] [Time:0.03630]
	[Generator][Epochs 10/50] [G epoch loss: -16.58776] [Time:0.03607]
	[Generator][Epochs 11/50] [G epoch loss: -16.43567] [Time:0.03261]
	[Generator][Epochs 12/50] [G epoch loss: -16.45572] [Time:0.03668]
	[Generator][Epochs 13/50] [G epoch loss: -16.56359] [Time:0.03676]
	[Generator][Epochs 14/50] [G epoch loss: -16.38649] [Time:0.03534]
	[Generator][Epochs 15/50] [G epoch loss: -16.20469] [Time:0.03699]
	[Generator][Epochs 16/50] [G epoch loss: -16.46661] [Time:0.03296]
	[Generator][Epochs 17/50] [G epoch loss: -16.38048] [Time:0.03324]
	[Generator][Epochs 18/50] [G epoch loss: -16.40488] [Time:0.03652]
	[Generator][Epochs 19/50] [G epoch loss: -16.67821] [Time:0.03472]
	[Generator][Epochs 20/50] [G epoch loss: -16.31845] [Time:0.03673]
	[Generator][Epochs 21/50] [G epoch loss: -16.42012